## Eletrical Usage

Take the optimal machine learning method for each independant variable:

- Location average income $\rightarrow$ kNN
- Weather $\rightarrow$ Random Forest
    - Rain [mm]
    - Wind speed [km/h]
    - Temperature [°C]
- Time for industrial activity $\rightarrow$ Regression
- Number of High-Rise buildings $\rightarrow$ kNN

In [ ]:
## Weather

import scikit-learn as sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

